In [6]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [7]:
park = Customer('Jack',100)
park

Customer(name='Jack', fidelity=100)

In [8]:
class LineItem:
    """구매할 물품/갯수 생성해서 총 가격을 반환"""
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [9]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체

    def total(self):
        """__total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        """할인금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self) # self = Order 객체
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [10]:
class Promotion(ABC): # strategy : abstract base class
    """할인 혜택 클래스들의 형태를 선언"""

    @abstractmethod # 이 클래스를 상속하는 클래스는 반드시 이 메서드를 선언해야함
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환"""
        pass

In [11]:
class FidelityPromo(Promotion):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""

    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [12]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]

In [13]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [14]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [15]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [16]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [17]:
long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)] # 물품 종류가 10개

In [18]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [19]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [20]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self) # promotion은 함수 객체
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [21]:
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [22]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [23]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [24]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [25]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [26]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
# 함수들로 구현된 전략들의 리스트

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [27]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [28]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [29]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [30]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'Order', '_i2', '_i3', '_i4', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i5', 'park', '_5', '_i6', '_i7', '_7', '_i8', 'LineItem', '_i9', '_i10', 'Promotion', '_i11', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i12', 'joe', 'ann', 'cart', '_i13', '_13', '_i14', '_14', '_i15', 'banana_cart', '_i16', '_16', '_i17', 'long_order', '_i18', '_18', '_i19', '_19', '_i20', '_i21', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i22', '_22', '_i23', '_23', '_i24', '_24', '_i25', '_25', '_i26', 'promos', 'best_promo', '_i27', '_27', '_i28', '_28', '_i29', '_29', '_i30'])

In [31]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [32]:
promos = [globals()[name] for name in globals()
         if name.endswith('_promo')
         and name != 'best_promo'] # 재귀호출을 피하기 위함
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [33]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [34]:
@deco
def target():
    print("running target()")

target()

running inner()


In [35]:
target

<function __main__.deco.<locals>.inner()>

In [36]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1() # 이 경우 register()가 원래 함수를 반환하므로 f1()이 실행됨
    f2()
    f3()

if __name__=='__main__':
    main()

running register(<function f1 at 0x7bd65249d360>)
running register(<function f2 at 0x7bd65249d3f0>)
running main()
registry -> [<function f1 at 0x7bd65249d360>, <function f2 at 0x7bd65249d3f0>]
running f1()
running f2()
running f3()


In [37]:
promos = []

def promotion(promo_func):
    """promotion() 데커레이터는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

In [38]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [39]:
b=6
f1(3)

3
6


In [40]:
b = 6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [41]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b=9

f3(3)

3
6


In [42]:
b

9

In [43]:
f3(3)

3
9


In [44]:
b=30
f3(3)

3
30


In [45]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [46]:
avg = Averager()
avg(10)

10.0

In [47]:
avg(11)

10.5

In [48]:
avg(12)

11.0

In [49]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [50]:
avg = make_averager()
avg(10)

10.0

In [51]:
avg(11)

10.5

In [52]:
avg(12)

11.0

In [53]:
avg.__code__.co_varnames

('new_value', 'total')

In [54]:
avg.__code__.co_freevars

('series',)

In [55]:
avg.__closure__

(<cell at 0x7bd63989df00: list object at 0x7bd639337f80>,)

In [56]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [57]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

In [58]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [59]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [60]:
avg = make_averager()
avg(10)

10.0

In [61]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [62]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12323526s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000078s] factorial(1) -> 1
[0.00003870s] factorial(2) -> 2
[0.00209289s] factorial(3) -> 6
[0.00218235s] factorial(4) -> 24
[0.00221016s] factorial(5) -> 120
[0.00223820s] factorial(6) -> 720
6! = 720


In [63]:
factorial.__name__

'clocked'

In [64]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args,**kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [65]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12327886s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00004172s] factorial(2) -> 2
[0.00006986s] factorial(3) -> 6
[0.00009632s] factorial(4) -> 24
[0.00012326s] factorial(5) -> 120
[0.00213075s] factorial(6) -> 720
6! = 720


In [66]:
factorial.__name__

'factorial'